# Библиотеки

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import os

import keras
from keras.models import Model, Sequential, model_from_json
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils

import cv2
from tqdm import tqdm

from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


# Загрузка и обработка данных

In [3]:
core_dir = 'Z:\\Kaggle Amazon Rainforest\\'
#core_dir = 'C:\\Kaggle\\Understanding the Amazon from Space\\'
train_dir = core_dir + 'train-jpg\\'
test_dir = core_dir + 'test-jpg\\'

cathegories = ['agriculture', 'artisinal_mine', 'bare_ground', 
                      'blooming', 'blow_down', 'clear', 'cloudy', 'conventional_mine', 
                      'cultivation', 'habitation', 'haze', 'partly_cloudy', 'primary', 
                      'road', 'selective_logging', 'slash_burn', 'water']

train_data = pd.read_csv(core_dir + 'train.csv')
train_data.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


# Функции

подготовка тренировочной выборки по имени категории
принимает категорию, возвращает массив из 0 и 1 (1 - файл содержит категорию, 0 - иначе)

In [4]:
def TrainingSetByCathegory(cathegory):
    df = pd.DataFrame()
    
    df['image_name'] = train_data.image_name.values
    df['indicator'] = np.zeros(train_data.shape[0])
    
    df.loc[df['image_name'].isin(cathegory_dict[cathegory]), ['indicator']] = 1
    
    return df.indicator.values

формирование полной (по всем категориям) тестовой выборки

In [5]:
def TestSetWhole(data, col_name = 'tags'):
    encoder = LabelEncoder()
    encoder.fit(cathegories)
    
    result = np.zeros((data.shape[0], len(cathegories)))
    
    for i in range(data.shape[0]):
        tag = data[col_name][i]
        tag = tag.split(' ')
        v = encoder.transform(tag)
        
        for j in v:
            result[i, j] = 1
    
    return(result)

функция предназначенная для конвертации .jpg изображения в numpy массив
если transparency = False, то каждая точка представляется 3 числами иначе - 4

In [6]:
def ImageToNumpy(img_name, img_dir = train_dir, img_type = '.jpg', transparency = False):
    img = Image.open(train_dir + img_name + img_type)
    img.load()
    data = np.asarray( img, dtype="int32" )
    img.close()
    
    if transparency:
        return data
    else:
        return data[:, :, 0:3]

функция для конвертации вектора компоненты которого принадлежат [0, 1] в вектор из 0 и 1 при заданном пороге (treshold)

In [7]:
def BinimialPrediction(x, treshold = 0.5):
    result = np.zeros(x.shape)
    
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if x[i, j] >= treshold:
                result[i, j] = 1
    
    return(result)

def FBettaScore(x_true, x_predicted, betta = 2):
    if len(x_true) == len(x_predicted):
        tp = 0
        fp = 0
        fn = 0

        for i in range(len(x_predicted)):
            if x_true[i] == 1 and x_predicted[i] == 1:
                tp += 1
            
            if x_true[i] == 0 and x_predicted[i] == 1:
                fp += 1
            
            if x_true[i] == 1 and x_predicted[i] == 0:
                fn += 1
        
        if tp == 0 or (tp + fp) == 0 or (tp + fn) == 0:
            return(0)
        else:
            precision = tp/(tp + fp)
            recall = tp/(tp + fn)
            
            #print(precision, recall)
            return((1 + betta**2)*precision*recall/(betta**2*precision + recall))
    else:
        print('FBettaScore error! len(x_true) != len(x_predicted)')

def AvgFBettaScore(x_true, x_predicted, betta = 2):
    result = 0
    n = x_true.shape[0]
    
    x_predicted = BinimialPrediction(x_predicted)
    
    for i in range(n):
        result += FBettaScore(x_true[i, :], x_predicted[i, :], betta)
    
    return(result/n)


# Вычисления

создадим словарь категорий
каждой категории будет соответствовать список файлов, в которых данная категория присутствует

In [8]:
cathegory_dict = {cathegory: [] for cathegory in cathegories}

ind = 0
for tag in train_data.tags.values:
    file_name = train_data.image_name[ind]
    
    cathegory_names_list = tag.split(' ')
    
    for cathegory in cathegory_names_list:
        cathegory_dict[cathegory].append(file_name)
    
    ind += 1   

формирование обучающей выборки

In [8]:
X = []

for img_name in tqdm(train_data.image_name.values):
    
    img = cv2.imread(train_dir + img_name + '.jpg')
    res = cv2.resize(img, (32, 32))
    X.append(res)
    
X = np.array(X, np.float16) / 255.

100%|████████████████████████████████████| 40479/40479 [15:40<00:00, 46.92it/s]


формирование тестовой выборки

In [43]:
Y = TestSetWhole(train_data)

(40479, 17)

разбиение выборки на обучение и тест

In [10]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Настройка и  обучение сети для всех категорий

In [11]:
batch_size = 128 # in each iteration, we consider 32 training examples at once
num_epochs = 1 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

In [12]:
# загрузка изображений и их приведение к подходящему для обработки виду
num_train = 40479
depth = 32 
height = 32
width = 3 

num_classes = len(cathegories)

#y_train = TrainingSetByCathegory(cathegories[0]) #y_train labels (для категории_0 1, если подходит, 0-иначе)
#Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
#X_train = X

In [ ]:
# переделать
'''
def f2_metric(y_true, y_pred):
    result = []
    for i in range(y_true.shape[0]):
        result.append(fbeta_score(y_true[i, :], y_pred[i, :], 2))
    
    result = np.mean(result)
    
    return(result)
'''

In [ ]:
inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras!

# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)

conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)

# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)

conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)

# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax

flat = Flatten()(drop_2)

hidden = Dense(hidden_size, activation='relu')(flat)

drop_3 = Dropout(drop_prob_2)(hidden)

out = Dense(num_classes, activation='sigmoid')(drop_3)

model = Model(input=inp, output=out) # To define a model, just specify its input and output layers

model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(x_train, y_train, # Train the model using the training set...
          batch_size=batch_size, nb_epoch=num_epochs,
          verbose=1, validation_split=0.2) # ...holding out 10% of the data for validation

#model.evaluate(x_test, y_test, verbose=1) # Evaluate the trained model on the test set!
cnn_prediction = model.predict(x_test)

In [ ]:
# Генерируем описание модели в формате json
model_json = model.to_json()
# Записываем модель в файл
json_file = open("model3.json", "w")
json_file.write(model_json)
json_file.close()

model.save_weights("model3.h5")

## Делаем предсказание

In [4]:
# load json and create model
json_file = open('Z:\\Kaggle Amazon Rainforest\\model2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights('Z:\\Kaggle Amazon Rainforest\\model2.h5')
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Loaded model from disk


функция для формирования тестовой выборки

In [ ]:
X_pred = []

for img_name in tqdm(os.listdir(test_dir)):
    if img_name.endswith("jpg"):
        img = cv2.imread(test_dir + img_name)
        res = cv2.resize(img, (32, 32))
        X_pred.append(res)
    else:
        print("This file is not jpg: " + img_name)
    
    
X_pred = np.array(X_pred, np.float16) / 255.

  4%|█▋                                   | 1792/40670 [00:36<24:29, 26.46it/s]

In [5]:
type(X_pred)

NameError: name 'X_pred' is not defined

In [13]:
img_prediction = loaded_model.predict(X_pred)

In [18]:
img_prediction[0]

array([  1.73672009e-03,   1.78920928e-07,   1.26425120e-05,
         1.01856154e-03,   4.09660803e-04,   9.96547878e-01,
         7.31760520e-06,   7.59083548e-07,   2.86426395e-04,
         5.33856917e-04,   3.20503418e-03,   5.02295326e-04,
         9.99997258e-01,   4.74329293e-03,   1.20196753e-04,
         2.18173810e-07,   3.13977823e-02], dtype=float32)

In [74]:
def test_csv_builder(model, csv_file_dir = core_dir + 'submission.csv'):
    submission_csv = open(csv_file_dir, 'w')
    submission_csv.write('image_name, tags\n')
           
    img_prediction = model.predict(X_pred)
        
    line = img_name + ','
    for i in range(len(cathegories)):
        if img_prediction[i] == 1:
            line += cathegories[i] + ' '
        
    submission_csv.write(line + '\n')
    
    submission_csv.close()

In [ ]:
csv_file_dir = core_dir + 'submission.csv'
submission_csv = open(csv_file_dir, 'w')
submission_csv.write('image_name, tags\n')
for img_name in tqdm(os.listdir(test_dir)):
    if img_name.endswith("jpg"):
        line = img_name + ','
        for img_pred in img_prediction:
            for i in range(len(cathegories)):
                if img_pred[i] >= 0.95:
                    line += cathegories[i] + ' '
submission_csv.write(line + '\n')
submission_csv.close()

  0%|                                     | 3/40670 [00:06<25:10:05,  2.23s/it]